In [1]:
import torch
from time import time
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
from zipfile import ZipFile
import numpy as np
from time import time
from torchvision import datasets
from torchvision import transforms
import pandas as pd
import numpy as np
import zipfile
import re
import os
from PIL import Image
from torchvision.io import read_image
from matplotlib import image
from prepare_dataset import MyDataset,subset_generator
from utils import display_some_images,stats_of_image_and_label,MyCuda_Stats,get_output_features_of_model
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from Model import Network,new_model
from train_fn import train,validation,image_validation
from tqdm import tqdm


In [2]:
fullvgg=full_vgg()
total_trainable_parameters(fullvgg)
total_trainable_parameters(fullvgg.features)
total_trainable_parameters(vgg_all_freezed())
total_trainable_parameters(vgg_layer_freeze(24))


Total trainable parameters of VGG is 	 138357544

Total trainable parameters of Sequential is 	 14714688

Total trainable parameters of Sequential is 	 0

Total trainable parameters of Sequential is 	 7079424


In [2]:
# DATA LOADER PARAMETERS
batch=16
CUDA_LAUNCH_BLOCKING=1 

In [3]:
MyCuda_Stats()

No of GPUs i have is 1
0
My Graphic Card is NVIDIA GeForce GTX 1050 Ti
Is Cuda Available True


In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])
target_transform=transforms.Compose([transforms.ToTensor()])

In [3]:
anno_dir=r"F:\Affectnet\annotations"
image_dir=r"F:\Affectnet\train_set\train_set\images"
train_data=MyDataset(anno_dir,image_dir,transform=transform, target_transform=None)

In [16]:
val_anno_dir=r"F:\Affectnet\val_set\val_set\annotations"
val_image_dir=r"F:\Affectnet\val_set\val_set\images"
val_data=MyDataset(val_anno_dir,val_image_dir,transform=transform, target_transform=None)

In [ ]:
num_classes = 7    
dataset = MyDataset1(num_classes)
labels = torch.zeros(num_classes, dtype=torch.long)

for _, target in dataset:
    labels += target

In [21]:
Subset_train_sampler=subset_generator(train_data,5000)

In [6]:
train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch,
                                         shuffle=False,
                                         num_workers=4,
                                         pin_memory=True,
                                         sampler=None)
val_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=batch,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)

In [7]:
print(len(train_loader))
print(len(val_loader))

17979
250


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [6]:
feature_extractor=vgg_layer_freeze(17)
total_trainable_parameters(feature_extractor)


Total trainable parameters of Sequential is 	 12979200


In [7]:

#input_features_for_denselayer=get_output_features_of_model(model=feature_extractor,batch_size=32,dataloader=train_loader)
input_features_for_denselayer=25088


In [8]:
model1=new_model(feature_extractor,input_features_for_denselayer).to(device)
model1

new_model(
  (extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dil

In [14]:
total_trainable_parameters(model1)


Total trainable parameters of new_model is 	 115776520


In [9]:
lr=0.0001
loss1=nn.CrossEntropyLoss()
optimizer=optim.Adam(model1.parameters(),lr)

In [11]:
model1.load_state_dict(torch.load(r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\modelrecent.pth"))

<All keys matched successfully>

In [12]:
optimizer.load_state_dict(torch.load(r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\optimizerrecent.pth"))

In [16]:
validation(val_loader,model1,loss1)

100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [02:28<00:00,  1.68it/s]

test error-2.136537 
 Accuracy-48.212053%


In [24]:
# NO OF EPOCHS IS 5
epochs=50
for epoch in range(epochs):
    print(f'Epoch-{epoch}')
    train(train_loader,model1,loss1,optimizer)
    validation(val_loader,model1,loss1)


Epoch-0


100%|██████████████████████████████████████████████████████████████████████████| 17979/17979 [3:58:40<00:00,  1.26it/s]


loss:0.015967 batch:17978/17979


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:49<00:00,  2.27it/s]


test error-1.883609 
 Accuracy-48.387097%
Epoch-1


...

loss:0.010771 batch:17978/17979


100%|████████████████████████████████████████████████████████████████████████████████| 250/250 [01:30<00:00,  2.77it/s]


test error-2.136538 
 Accuracy-48.212053%
Epoch-2


  0%|                                                                                        | 0/17979 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [25]:
torch.save(model1.state_dict(), r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\modelrecent.pth")

In [26]:
torch.save(optimizer.state_dict(), r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\optimizerrecent.pth")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device='cuda', abbreviated=True)